In [50]:
%pip -q install google-genai

In [51]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [52]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

from IPython.display import display, HTML, Markdown

In [53]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [69]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
import re # Para usar expressões regulares

warnings.filterwarnings("ignore")

In [55]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [56]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [67]:
###########################################################
# --- Agente 1: Sugestão de Componentes --- #
###########################################################
def agente_sugestao_componentes(descricao_trabalho):

    especialista_hardware = Agent(
        name="agente_especialista_hardware",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um arquiteto de sistemas de hardware especializado em notebooks.
        Considerando as tecnologias e componentes mais recentes disponíveis, e com base na seguinte descrição do uso do notebook:
        {descricao_trabalho}, quais são as configurações de componentes recomendadas (processador, memória RAM, armazenamento, placa de vídeo, se aplicável),
        priorizando configurações com bom custo-benefício para o usuário.Considere também as restrições orçamentárias do usuário, caso sejam fornecidas na descrição do trabalho.
        Forneça uma breve justificativa para a escolha de cada componente recomendado.
        Instrução adicional: Se a descrição do trabalho não fornecer informações suficientes para uma recomendação direta ou se você precisar de informações mais
        amplas sobre as configurações ideais, utilize a ferramenta do google (google_search) para complementar seu conhecimento e fornecer a melhor recomendação possível.
        Inclua na sua resposta as fontes consultadas, se houver.
        """,
        description="Agente especializado em sugerir configurações de hardware para notebooks.",
        tools=[google_search]
    )

    entrada_do_agente = f"Descrição do trabalho: {descricao_trabalho}"
    recomendacoes = call_agent(especialista_hardware, entrada_do_agente)
    return recomendacoes

In [100]:
###########################################################
# --- Agente 2: Buscador de Notebooks --- #
###########################################################
def agente_buscador_notebooks(recomendacoes_componentes):

    buscador_notebooks = Agent(
        name="agente_buscador_notebooks",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um assistente de pesquisa altamente especializado em encontrar notebooks com configurações específicas disponíveis para compra no Brasil.
        Sua tarefa é usar a ferramenta de busca do Google (google_search) para encontrar até 10 notebooks disponíveis para compra no Brasil
        que correspondam o mais precisamente possível às seguintes configurações recomendadas: {recomendacoes_componentes}.
        Ao realizar a busca, inclua termos como "Brasil" ou nomes de lojas online populares no Brasil para restringir os resultados.
        Seja flexível para considerar pequenas variações nas especificações, como uma quantidade maior de RAM ou um modelo de processador ligeiramente superior,
        caso a configuração exata não esteja amplamente disponível no mercado brasileiro.

        Para cada notebook encontrado, liste o link da página do produto e forneça um breve resumo, **iniciando com o nome/modelo do notebook**,
        seguido das suas principais configurações, destacando aquelas que correspondem às recomendações. Mencione claramente a disponibilidade no Brasil
        e priorize resultados de lojas online com boa reputação no país. Se disponível, mencione brevemente as avaliações de outros usuários sobre os
        notebooks encontrados em sites brasileiros.
        """,
        description="Agente especializado em buscar notebooks com configurações específicas no Google, com foco no mercado brasileiro.",
        tools=[google_search],
    )

    entrada_do_agente = f"Recomendações de componentes: {recomendacoes_componentes}"
    resultados_busca = call_agent(buscador_notebooks, entrada_do_agente)
    return resultados_busca

In [71]:
###########################################################
# --- Agente 3: Analisador de Reviews --- #
###########################################################
def agente_analisador_reviews(nome_notebook):

    analisador_reviews = Agent(
        name="agente_analisador_reviews",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um assistente de pesquisa especializado em analisar reviews de notebooks.
        Sua tarefa é usar a ferramenta de busca do Google (google_search) para encontrar reviews e análises de fontes confiáveis
        (incluindo vídeos no YouTube e artigos de sites especializados) sobre o seguinte notebook: {nome_notebook}.

        Com base nessas análises, produza um relatório conciso listando os principais pontos positivos e negativos do notebook.
        Para cada ponto, tente inferir a frequência com que é mencionado nas diversas fontes. Apresente os pontos de forma clara e concisa (usando marcadores, por exemplo).
        Se a fonte do review especificar o público-alvo (ex: gamers, profissionais), mencione essa informação junto ao ponto levantado.
        Foque em aspectos como desempenho, bateria, tela, teclado, qualidade de construção e outros fatores relevantes para a experiência do usuário.

        Apresente o resultado final como uma lista de notebooks no seguinte formato:

        ```
        [
          {{
            "Nome": "{nome_notebook}",
            "Pontos Positivos": ["Exemplo de ponto positivo 1", "Exemplo de ponto positivo 2"],
            "Pontos Negativos": ["Exemplo de ponto negativo 1", "Exemplo de ponto negativo 2"]
          }}
        ]
        ```
        """,
        description="Agente especializado em analisar reviews de notebooks e identificar pontos positivos e negativos.",
        tools=[google_search],
    )

    entrada_do_agente = f"Nome do notebook a ser analisado: {nome_notebook}"
    relatorio_reviews = call_agent(analisador_reviews, entrada_do_agente)
    return relatorio_reviews

In [101]:
###########################################################
# --- Agente 4: Escolha por Custo-Benefício --- #
###########################################################
def agente_escolha_custo_beneficio(descricao_trabalho, analise_notebooks):

    escolhedor_custo_beneficio = Agent(
        name="agente_escolhedor_custo_beneficio",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em avaliação de notebooks e custo-benefício.
        Dada a descrição original do trabalho do usuário: {descricao_trabalho},
        e considerando a seguinte análise dos notebooks encontrados:

        {analise_notebooks}

        Sua tarefa é analisar essas informações e identificar o notebook que oferece o melhor custo-benefício para atender às necessidades do usuário descritas no trabalho original.
        Justifique sua escolha de forma clara e focada nos benefícios práticos para o usuário.
        Explique como os pontos positivos do notebook selecionado se traduzem em vantagens concretas para o tipo de trabalho que ele pretende realizar
        e como os pontos negativos são aceitáveis considerando o preço (se disponível na análise) e as outras opções. Aprimeira linha do relatório deve ser o nome do notebook.
        """,
        description="Agente especializado em escolher o melhor notebook com base no custo-benefício para o usuário.",
    )

    entrada_do_agente = f"Descrição do trabalho: {descricao_trabalho}\nAnálise dos notebooks: {analise_notebooks}"
    melhor_notebook = call_agent(escolhedor_custo_beneficio, entrada_do_agente)
    return melhor_notebook

In [73]:
###########################################################
# --- Agente 5: Gerador de Relatório Final com Preço --- #
###########################################################
def agente_relatorio_final(notebook_escolhido, justificativa):

    gerador_relatorio = Agent(
        name="agente_gerador_relatorio",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um assistente de apresentação de resultados. Sua tarefa é pegar a seguinte recomendação de notebook e a justificativa da escolha
        e formatá-las em um relatório conciso e fácil de entender para o usuário. Além disso, você deve usar a ferramenta de busca do Google (google_search)
        para pesquisar o preço atual do notebook recomendado: {notebook_escolhido}.

        **Notebook Escolhido:** {notebook_escolhido}
        **Justificativa:** {justificativa}

        O relatório deve incluir o nome do notebook escolhido, a justificativa detalhada explicando os benefícios para o tipo de trabalho do usuário
        e considerando o custo-benefício, e o preço atual encontrado na pesquisa. Apresente as informações de forma organizada e utilizando uma linguagem clara e acessível.
        Inclua também um link para onde o preço foi encontrado, se possível.
        """,
        description="Agente especializado em gerar um relatório final para o usuário, incluindo o preço do notebook.",
        tools=[google_search],
    )

    entrada_do_agente = f"Notebook Escolhido: {notebook_escolhido}\nJustificativa: {justificativa}\nBuscar preço de: {notebook_escolhido}"
    relatorio_usuario = call_agent(gerador_relatorio, entrada_do_agente)
    return relatorio_usuario

In [75]:
###########################################################
# --- Agente para Tirar Dúvidas sobre o Notebook --- #
###########################################################
def agente_responde_duvida(info_notebook, pergunta_usuario):

    respondedor_duvidas = Agent(
        name="agente_responde_duvida",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um assistente especializado em responder a dúvidas sobre o seguinte notebook recomendado:

        **Informações do Notebook:** {info_notebook}

        Com base nessas informações, tente responder da forma mais clara e concisa possível à seguinte pergunta do usuário:

        **Pergunta:** {pergunta_usuario}

        Se a resposta para a pergunta não puder ser encontrada diretamente nas informações fornecidas, você pode usar a ferramenta de busca do Google (google_search) para procurar em fontes confiáveis (como sites de fabricantes, reviews de especialistas) por uma resposta.

        Ao usar a busca no Google, siga estas diretrizes estritamente:
        1. **Cite suas fontes:** Mencione claramente de onde você obteve a informação.
        2. **Evite inventar:** Não crie informações que não foram encontradas nas fontes.
        3. **Foco na pergunta:** Responda diretamente à pergunta do usuário.
        4. **Seja conciso:** Mantenha a resposta breve e objetiva.

        Se, após a busca, você ainda não encontrar uma resposta confiável, informe ao usuário que a informação não está disponível no momento.
        """,
        description="Agente especializado em responder a dúvidas sobre o notebook recomendado, com capacidade de buscar informações no Google.",
        tools=[google_search],
    )

    entrada_do_agente = f"Informações do Notebook: {info_notebook}\nPergunta do Usuário: {pergunta_usuario}"
    resposta = call_agent(respondedor_duvidas, entrada_do_agente)
    return resposta

In [91]:
###########################################################
# --- Agente para Validar a Descrição do Trabalho --- #
###########################################################
def agente_validador_descricao(descricao_trabalho):

    validador = Agent(
        name="agente_validador_descricao",
        model="gemini-2.0-flash",
        instruction=f"""
        Sua tarefa é verificar se a seguinte descrição de como o usuário pretende usar o notebook faz sentido dentro do contexto de usos comuns para um notebook:

        **Descrição do Trabalho:** {descricao_trabalho}

        Analise a descrição e determine se ela representa uma atividade ou necessidade plausível para o uso de um notebook.

        - Se a descrição fizer sentido, retorne uma string contendo '1'.
        - Se a descrição não fizer sentido ou for vaga demais, retorne uma sugestão de descrição mais clara e concisa, simples, fornecendo um exemplo simples de como
        o usuário poderia descrever o uso do notebook. Retorne somente o texto do exemplo de sugestão sugestão.
        """,
        description="Agente especializado em validar se a descrição do trabalho do usuário faz sentido.",
    )

    entrada_do_agente = f"Descrição do Trabalho: {descricao_trabalho}"
    resultado_validacao = call_agent(validador, entrada_do_agente)
    return resultado_validacao

In [98]:
###########################################################
# --- Agente para Comparar Notebooks --- #
###########################################################
def agente_comparador_notebooks(nome_notebook1, nome_notebook2):
    """
    Agente que compara as especificações e características de dois notebooks.

    Args:
        nome_notebook1 (str): O nome do primeiro notebook.
        nome_notebook2 (str): O nome do segundo notebook.

    Returns:
        str: A comparação formatada dos dois notebooks.
    """
    comparador = Agent(
        name="agente_comparador_notebooks",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em comparação de notebooks. Sua tarefa é receber os nomes de dois notebooks e apresentar uma comparação lado a lado das suas principais especificações e características.

        Os notebooks a serem comparados são:

        Notebook 1: {nome_notebook1}
        Notebook 2: {nome_notebook2}

        Para cada notebook, liste as seguintes informações, se disponíveis:

        - Processador
        - Memória RAM
        - Armazenamento (tipo e capacidade)
        - Placa de Vídeo (se aplicável)
        - Tamanho da Tela
        - Preço (se disponível)
        - Um breve resumo das opiniões dos usuários (pontos positivos e negativos, se disponíveis nas análises).

        Apresente a comparação em um formato fácil de ler, destacando as diferenças e semelhanças entre os dois modelos para ajudar o usuário a tomar uma decisão informada.
        """,
        description="Agente especializado em comparar as especificações e características de dois notebooks.",
    )

    entrada_do_agente = f"Comparar {nome_notebook1} com {nome_notebook2}."
    comparacao = call_agent(comparador, entrada_do_agente)
    return comparacao

In [105]:
print("💻 Iniciando o Sistema de Recomendação de Notebooks 💻")

# --- Loop para garantir que a primeira descrição do trabalho seja válida ---
while True:
    # Obter a Descrição do Trabalho do Usuário
    descricao_trabalho = input("❓ Por favor, descreva como você pretende usar o notebook: ")
    # Verifica se a descrição do trabalho não está vazia após remover espaços em branco.
    if descricao_trabalho.strip():
        # Chama o agente validador para verificar se a descrição faz sentido.
        resultado_validacao = agente_validador_descricao(descricao_trabalho)
        if resultado_validacao.strip() == "1":
            print(f"Ótimo! Analisando as melhores opções para '{descricao_trabalho}'...")
            break # Sai do loop se a descrição for considerada válida.
        else:
            print(f"⚠️ A descrição fornecida pode não fazer sentido. Uma sugestão seria: '{resultado_validacao}'. Por favor, tente novamente.")
    else:
        print("⚠️ Por favor, forneça uma descrição válida do seu trabalho.")

# Inserir lógica do sistema de agentes ################################################
# --- Agente 1: Sugestão de Componentes ---
print("\n⚙️ Iniciando análise das configurações ideais de hardware...")
# Este agente recebe a descrição do trabalho do usuário e retorna uma sugestão de configurações de hardware.
recomendacoes_componentes = agente_sugestao_componentes(descricao_trabalho)
print("\n--- Sugestão de Componentes ---\n")
display(Markdown(recomendacoes_componentes)) # Exibe a resposta do agente formatada em Markdown.
print("-------------------------------------------------------------------")

# --- Agente 2: Buscador de Notebooks ---
print("\n🔍 Buscando notebooks que sejam compatíveis com às configurações recomendadas...")
# Este agente recebe as recomendações de componentes e busca por notebooks que atendam a essas especificações.
notebooks_encontrados = agente_buscador_notebooks(recomendacoes_componentes)

# --- Extrair Nomes dos Notebooks e Chamar Agente 3 ---
nomes_notebooks = []
# Itera sobre cada linha da resposta do Agente 2 para extrair os nomes dos notebooks encontrados.
for linha in notebooks_encontrados.splitlines():
    # Utiliza uma expressão regular para encontrar linhas que começam com um número, ponto, espaço e um nome de notebook em negrito.
    if re.match(r'^\d+\.\s+\*\*(.+?)\*\*', linha):
        # Extrai o nome do notebook utilizando outra expressão regular.
        nome = re.findall(r'^\d+\.\s+\*\*(.+?)\*\*', linha)[0]
        # Adiciona o nome do notebook à lista de nomes.
        nomes_notebooks.append(nome)

analises_notebooks = []
# Itera sobre a lista de nomes de notebooks encontrados para chamar o Agente 3 para cada um.
print("\n🧐 Analisando as opiniões dos usuários sobre os notebooks encontrados...")
for nome_notebook in nomes_notebooks:
    # --- Agente 3: Analisador de Reviews ---
    # Este agente recebe o nome de um notebook e busca e resume reviews sobre ele.
    analise = agente_analisador_reviews(nome_notebook)
    # Adiciona a análise do notebook à lista de análises.
    analises_notebooks.append(analise)

# --- Agente 4: Escolha por Custo-Benefício ---
print("\n💰 Identificando a melhor opção de custo-benefício...")
# Este agente analisa as reviews dos notebooks e as recomendações de componentes para escolher a melhor opção custo-benefício.
melhor_notebook_info = agente_escolha_custo_beneficio(descricao_trabalho, analises_notebooks)

# --- Agente 5: Gerador de Relatório Final ---
print("\n📝 Gerando o relatório final com a recomendação...")
# Este agente recebe as informações do melhor notebook escolhido e gera um relatório final para o usuário.
# Por simplicidade, a mesma informação está sendo passada para ambos os argumentos.
# Em uma implementação mais completa, o primeiro argumento poderia ser informações gerais e o segundo detalhes específicos.
relatorio_final = agente_relatorio_final(melhor_notebook_info, melhor_notebook_info)
display(Markdown(relatorio_final)) # Exibe o relatório final formatado em Markdown.
print("-------------------------------------------------------------------")

print("\n✨ Relatório de Recomendação de Notebook Gerado! ✨")

# --- Interação com o Usuário após a Recomendação ---

# Cria uma lista contendo o notebook recomendado como a melhor opção
# e os nomes de outros notebooks que foram considerados durante a busca.
notebooks_sugeridos = [melhor_notebook_info] + nomes_notebooks

# Inicia um loop contínuo para interagir com o usuário até que ele esteja satisfeito ou decida encerrar.
while True:
    # Pergunta ao usuário sobre sua satisfação com a recomendação atual,
    # oferecendo as opções de 'sim' (satisfeito), 'não' (insatisfeito, deseja nova recomendação),
    # 'dúvida' (tem perguntas sobre o notebook recomendado) e 'comparar' (deseja comparar com outras opções).
    satisfacao = input("❓ Você está satisfeito com a recomendação? (sim/não/dúvida/comparar): ").lower()

    # Se o usuário responder que está satisfeito.
    if satisfacao == "sim":
        print("\n👍 Ótimo! Ficamos felizes em ajudar.")
        break # Sai do loop de interação, encerrando o sistema.

    # Se o usuário responder que não está satisfeito e deseja uma nova recomendação.
    elif satisfacao == "não":
        print("\n🔄 Entendi. Por favor, descreva o que não te agradou ou quais são suas novas preferências para que eu possa gerar outra recomendação:")
        nova_descricao = input() # Solicita ao usuário uma nova descrição de suas necessidades.
        # Verifica se o usuário forneceu uma nova descrição (não deixou em branco).
        if nova_descricao:
            print(f"\n✨ Analisando suas novas preferências: '{nova_descricao}'...")
            # Chama o agente para sugerir componentes ideais com base na nova descrição.
            recomendacoes_componentes = agente_sugestao_componentes(nova_descricao)
            print("\n--- Nova Sugestão de Componentes ---\n")
            display(Markdown(recomendacoes_componentes))
            print("-------------------------------------------------------------------")

            print("\n🔍 Buscando notebooks que sejam compatíveis com às configurações recomendadas...")
            # Chama o agente para buscar notebooks com as novas configurações recomendadas.
            notebooks_encontrados = agente_buscador_notebooks(recomendacoes_componentes)
            # Extrai os nomes dos notebooks encontrados na nova busca utilizando expressões regulares.
            nomes_notebooks = [re.findall(r'^\d+\.\s+\*\*(.+?)\*\*', linha)[0] for linha in notebooks_encontrados.splitlines() if re.match(r'^\d+\.\s+\*\*(.+?)\*\*', linha)]
            # Chama o agente para analisar as reviews dos notebooks encontrados na nova busca.
            analises_notebooks = [agente_analisador_reviews(nome_notebook) for nome_notebook in nomes_notebooks]
            # Chama o agente para escolher a melhor opção de custo-benefício com base nas novas análises.
            melhor_notebook_info = agente_escolha_custo_beneficio(nova_descricao, analises_notebooks)
            print("\n💰 Identificando a melhor opção de custo-benefício...")
            print("\n📝 Gerando o relatório final com a recomendação...")
            # Chama o agente para gerar um novo relatório final com a nova recomendação.
            relatorio_final = agente_relatorio_final(melhor_notebook_info, melhor_notebook_info)
            display(Markdown(relatorio_final))
            print("-------------------------------------------------------------------")
            print("\n✨ Novo Relatório de Recomendação de Notebook Gerado! ✨")
            # Atualiza a lista de notebooks sugeridos com a nova melhor opção e os outros notebooks encontrados.
            notebooks_sugeridos = [melhor_notebook_info] + nomes_notebooks
        else:
            print("\n⚠️ Por favor, forneça uma descrição para a nova recomendação.")

    # Se o usuário responder que tem dúvidas sobre o notebook recomendado.
    elif satisfacao == "dúvida":
        while True:
            pergunta = input("❓ Qual a sua dúvida sobre o notebook recomendado? (ou digite 'voltar' para sair) ")
            # Permite ao usuário sair do modo de dúvidas digitando 'voltar'.
            if pergunta.lower() == "voltar":
                break # Sai do loop de perguntas.
            # Se o usuário digitar uma pergunta.
            elif pergunta:
                print(f"\n🤔 Buscando informações sobre sua dúvida: '{pergunta}'...")
                # Chama o agente para responder à pergunta do usuário sobre o notebook recomendado.
                resposta_duvida = agente_responde_duvida(melhor_notebook_info, pergunta)
                print("\n--- Resposta para sua dúvida ---\n")
                display(Markdown(resposta_duvida))
                print("\n-------------------------------\n")
            # Se o usuário não digitar nada (apenas pressionar Enter).
            else:
                print("\n⚠️ Por favor, digite sua dúvida.")

    # Se o usuário responder que deseja comparar o notebook recomendado com outras opções.
    elif satisfacao == "comparar":
        # Verifica se há outros notebooks sugeridos além do melhor.
        if len(nomes_notebooks) < 1:
            print("\n⚠️ Não há outros notebooks suficientes para comparar com a recomendação atual.")
        else:
            # Tenta extrair o nome do melhor notebook assumindo que ele está na primeira linha da sua representação em string.
            nome_melhor_notebook = melhor_notebook_info.split('\n')[0].strip()
            print(f"\n💻 Selecione um notebook para comparar com '{nome_melhor_notebook}':")
            # Lista os outros notebooks sugeridos (excluindo o melhor) para o usuário escolher qual comparar.
            for i, nome in enumerate(nomes_notebooks):
                print(f"{i+1}. {nome}")
            try:
                opcao = int(input(f"Digite o número do notebook para comparar (1-{len(nomes_notebooks)}, ou 0 para voltar): "))
                # Verifica se a opção digitada é um número válido dentro do intervalo dos outros notebooks sugeridos.
                if 1 <= opcao <= len(nomes_notebooks):
                    notebook_comparar = nomes_notebooks[opcao - 1]
                    print(f"\n🔄 Comparando '{nome_melhor_notebook}' com '{notebook_comparar}'...")
                    # Chama o agente para realizar a comparação entre o melhor notebook e o escolhido pelo usuário.
                    comparacao = agente_comparador_notebooks(melhor_notebook_info, notebook_comparar)
                    print("\n--- Comparação de Notebooks ---\n")
                    display(Markdown(comparacao))
                    print("\n-------------------------------\n")
                # Se o usuário digitar 0, volta para a pergunta de satisfação.
                elif opcao == 0:
                    continue
                # Se a opção digitada não for válida.
                else:
                    print("\n⚠️ Opção inválida.")
            # Trata o erro caso o usuário digite algo que não seja um número.
            except ValueError:
                print("\n⚠️ Por favor, digite um número válido.")

    # Se o usuário não digitar uma das opções válidas ('sim', 'não', 'dúvida', 'comparar').
    elif not satisfacao.strip():
        print("\n⚠️ Por favor, digite 'sim', 'não', 'dúvida' ou 'comparar'.")
    # Para qualquer outra entrada inválida.
    else:
        print("\n⚠️ Resposta inválida. Por favor, digite 'sim', 'não', 'dúvida' ou 'comparar'.")

# Mensagem de agradecimento ao usuário após encerrar a interação.
print("\n👋 Obrigado por utilizar nosso sistema de recomendação!")

💻 Iniciando o Sistema de Recomendação de Notebooks 💻
❓ Por favor, descreva como você pretende usar o notebook: enegenheiro eletricista.
Ótimo! Analisando as melhores opções para 'enegenheiro eletricista.'...

⚙️ Iniciando análise das configurações ideais de hardware...

--- Sugestão de Componentes ---



Para um engenheiro eletricista, um notebook com bom custo-benefício deve equilibrar poder de processamento para simulações e cálculos, capacidade de executar softwares de engenharia, portabilidade para trabalho em campo e durabilidade.

Aqui está uma configuração recomendada com justificativas:

*   **Processador:** AMD Ryzen 5 7530U.

    *   **Justificativa:** Este processador oferece um bom equilíbrio entre desempenho e consumo de energia, ideal para tarefas exigentes como simulações e modelagem, sem esgotar rapidamente a bateria. A arquitetura Zen 3+ da AMD oferece um bom desempenho multitarefa e single-core, essencial para softwares de engenharia.
*   **Memória RAM:** 16GB DDR4 3200MHz.

    *   **Justificativa:** 16GB é o ideal para lidar com múltiplos programas de engenharia abertos simultaneamente, além de permitir trabalhar com arquivos grandes e simulações complexas sem lentidão. DDR4 oferece um bom custo-benefício em relação às tecnologias mais recentes.
*   **Armazenamento:** SSD de 512GB NVMe PCIe Gen3 ou Gen4.

    *   **Justificativa:** Um SSD NVMe garante tempos de inicialização rápidos para o sistema operacional e programas, além de carregamento e salvamento ágil de arquivos. 512GB oferece espaço suficiente para o sistema operacional, softwares de engenharia e arquivos de trabalho importantes. PCIe Gen4 oferece velocidades mais rápidas, mas PCIe Gen3 é uma opção mais acessível e ainda muito eficiente.
*   **Placa de Vídeo:** Placa de vídeo integrada AMD Radeon Graphics (integrada ao processador Ryzen).

    *   **Justificativa:** Para um engenheiro eletricista, uma placa de vídeo dedicada geralmente não é essencial, a menos que o trabalho envolva modelagem 3D complexa ou renderização intensiva. As placas integradas AMD Radeon Graphics oferecem desempenho suficiente para a maioria das tarefas de engenharia 2D/3D e visualização de dados, contribuindo para a economia de energia e redução de custos.

Para complementar estas informações, realizei uma pesquisa no Google utilizando as seguintes palavras chaves:


Com base nas informações encontradas nas pesquisas, aqui estão algumas opções adicionais e alternativas para a configuração recomendada, levando em conta o custo-benefício:

*   **Alternativa de Processador:** Intel Core i5 (11ª ou 12ª geração). Uma opção da Intel pode ser mais adequada dependendo dos softwares específicos que o engenheiro eletricista utiliza, já que alguns podem ser otimizados para processadores Intel.
*   **Alternativa de Placa de Vídeo:** Se o trabalho envolver tarefas de modelagem 3D ou uso intensivo de softwares gráficos, considerar uma placa de vídeo dedicada de entrada, como NVIDIA GeForce GTX 1650 ou RTX 3050, pode ser vantajoso.
*   **Marcas e Modelos Sugeridos:** Acer Nitro 5, Lenovo IdeaPad Gaming 3i, e Dell G15 são frequentemente recomendados como notebooks com bom custo-benefício para engenheiros.

**Considerações Adicionais:**

*   **Tela:** Uma tela de 15.6 polegadas Full HD (1920x1080) é recomendada para um bom equilíbrio entre portabilidade e conforto visual.
*   **Sistema Operacional:** Windows 11 é o sistema operacional mais recente e geralmente oferece boa compatibilidade com softwares de engenharia.
*   **Bateria:** Procurar por modelos com boa autonomia de bateria, especialmente se o trabalho exigir mobilidade.
*   **Refrigeramento:** Sistemas de resfriamento eficientes são importantes para evitar o superaquecimento, principalmente ao executar tarefas exigentes por longos períodos.

Lembre-se que o "melhor" custo-benefício depende das necessidades específicas de cada engenheiro e do orçamento disponível. É sempre recomendável pesquisar e comparar diferentes modelos antes de tomar uma decisão final.



-------------------------------------------------------------------

🔍 Buscando notebooks que sejam compatíveis com às configurações recomendadas...

🧐 Analisando as opiniões dos usuários sobre os notebooks encontrados...

💰 Identificando a melhor opção de custo-benefício...

📝 Gerando o relatório final com a recomendação...


Para fornecer um relatório conciso e útil, preciso que você especifique o nome do notebook escolhido e a justificativa detalhada para essa escolha. Sem essas informações, não consigo gerar o relatório solicitado. Além disso, preciso do nome exato do notebook para buscar o preço atual. Por favor, forneça esses detalhes para que eu possa completar a tarefa.



-------------------------------------------------------------------

✨ Relatório de Recomendação de Notebook Gerado! ✨
❓ Você está satisfeito com a recomendação? (sim/não/dúvida/comparar): não

🔄 Entendi. Por favor, descreva o que não te agradou ou quais são suas novas preferências para que eu possa gerar outra recomendação:
❓ Por favor, descreva como você pretende usar o notebook: um notebook mais barato.
enegenheiro eletricista. um notebook mais barato.
⚠️ A descrição fornecida pode não fazer sentido. Uma sugestão seria: 'engenheiro eletricista usando notebook para projetos de instalações elétricas e planilhas de custos.
'. Por favor, tente novamente.
❓ Por favor, descreva como você pretende usar o notebook: ngenheiro eletricista usando notebook para projetos de instalações elétricas e planilhas de custos.
enegenheiro eletricista. ngenheiro eletricista usando notebook para projetos de instalações elétricas e planilhas de custos.
Ótimo! Analisando as melhores opções para 'enegenheiro 

Para um engenheiro eletricista que utiliza o notebook para projetos de instalações elétricas e planilhas de custos, recomendo as seguintes configurações de componentes, priorizando um bom custo-benefício:


Com base nas informações coletadas e considerando o uso para projetos de instalações elétricas e planilhas de custos, sugiro a seguinte configuração com foco em custo-benefício:

*   **Processador:** Intel Core i5 (11ª geração ou superior) ou AMD Ryzen 5 (série 5000 ou superior).
    *   **Justificativa:** Processadores i5 ou Ryzen 5 oferecem um bom equilíbrio entre desempenho e preço. Eles são capazes de lidar com softwares de projeto elétrico (como AutoCAD, Revit - se usado para modelagem BIM - e outros) e planilhas sem grandes dificuldades.
*   **Memória RAM:** 16 GB DDR4 (mínimo 8GB).
    *   **Justificativa:** 16 GB de RAM é recomendado para garantir que você possa executar vários programas simultaneamente e trabalhar com arquivos grandes sem lentidão. Se o orçamento for limitado, 8GB é o mínimo aceitável, mas o ideal é priorizar 16GB.
*   **Armazenamento:** SSD de 512 GB (mínimo 256GB).
    *   **Justificativa:** Um SSD (Solid State Drive) oferece tempos de carregamento muito mais rápidos para o sistema operacional, aplicativos e arquivos, o que aumenta a produtividade. 512 GB é um bom tamanho para armazenar projetos, programas e documentos. Se o orçamento for apertado, um SSD de 256 GB pode ser suficiente, mas considere um SSD de maior capacidade no futuro.
*   **Placa de Vídeo:** Dedicada NVIDIA GeForce GTX 1650 (4GB) ou similar AMD Radeon RX.
    *   **Justificativa:** Uma placa de vídeo dedicada, mesmo que de entrada, pode melhorar o desempenho em softwares de modelagem 2D/3D e garantir uma experiência mais fluida ao trabalhar com projetos complexos. Modelos como a GTX 1650 são boas opções de custo-benefício. Se o trabalho for majoritariamente 2D, uma placa integrada (como a Intel Iris Xe Graphics ou AMD Radeon Graphics) pode ser suficiente, economizando no custo total.
*   **Tela:** 15.6 polegadas, resolução Full HD (1920x1080)
    *   **Justificativa:** Uma tela de 15.6 polegadas é um bom tamanho para trabalhar com projetos e planilhas. A resolução Full HD oferece uma boa qualidade de imagem e espaço de trabalho.
* **Bateria:** Priorizar modelos com boa autonomia, dependendo da necessidade de mobilidade.

**Observações:**

*   **Softwares:** Certifique-se de verificar os requisitos de sistema recomendados pelos softwares que você utiliza para garantir a compatibilidade e o desempenho ideal.
*   **Custo-benefício:** As configurações acima visam um bom equilíbrio entre desempenho e preço. É possível encontrar notebooks com essas características em diferentes faixas de preço, dependendo da marca, modelo e recursos adicionais.

**Exemplos de modelos que se encaixam nesse perfil (pesquisa em 17 de Maio de 2025):**

* Acer Nitro 5
* Lenovo Ideapad Gaming 3i
* Notebook Gamer Legion 5

Lembre-se de pesquisar e comparar preços antes de tomar uma decisão final.



-------------------------------------------------------------------

🔍 Buscando notebooks que sejam compatíveis com às configurações recomendadas...

💰 Identificando a melhor opção de custo-benefício...

📝 Gerando o relatório final com a recomendação...


Compreendo. Para gerar o relatório sobre o notebook ideal para um engenheiro eletricista, preciso primeiro de uma recomendação específica e suas respectivas especificações. Como não há nenhuma análise ou sugestão de notebook fornecida, não consigo criar o relatório completo com justificativa e preço.

Para prosseguir, preciso que você me forneça as seguintes informações:

1.  **Nome do Notebook:** Qual notebook você gostaria que eu recomendasse?
2.  **Especificações Técnicas:** Quais são as especificações desse notebook (processador, RAM, armazenamento, placa de vídeo, etc.)?
3.  **Justificativa Detalhada:** Por que este notebook é adequado para um engenheiro eletricista que trabalha com projetos de instalações elétricas e planilhas de custos?
4.  **Onde Buscar o Preço:** Qual o nome do notebook que devo procurar o preço?

Assim que tiver essas informações, poderei usar a ferramenta de busca para encontrar o preço atual e montar o relatório completo para você.

Ok, entendi. Para criar um relatório útil e preciso, preciso de informações específicas sobre o notebook que você deseja recomendar. Como você não forneceu detalhes sobre um notebook específico, não posso gerar o relatório agora.

Por favor, forneça as seguintes informações para que eu possa completar o relatório:

*   **Nome do Notebook:** O nome exato do modelo do notebook (ex: Dell XPS 15, MacBook Pro 16, Lenovo ThinkPad P1, etc.).
*   **Especificações Técnicas:** Detalhes sobre o processador (ex: Intel Core i7, AMD Ryzen 7), quantidade de RAM (ex: 16GB, 32GB), armazenamento (ex: 512GB SSD, 1TB SSD), placa de vídeo (ex: NVIDIA GeForce RTX 3050, AMD Radeon Pro), e tamanho da tela.
*   **Justificativa:** Uma explicação detalhada de por que este notebook é uma boa escolha para um engenheiro eletricista, considerando suas necessidades de software (ex: AutoCAD, softwares de simulação elétrica), tarefas (ex: projetos de instalações elétricas, planilhas de custos), e portabilidade.
*   **Onde Buscar o Preço:** O nome do modelo do notebook para buscar o preço.

Com essas informações, posso pesquisar o preço atual do notebook e criar um relatório completo e útil para você.



-------------------------------------------------------------------

✨ Novo Relatório de Recomendação de Notebook Gerado! ✨


KeyboardInterrupt: Interrupted by user